In [5]:
#APPROACH 1 #https://www.thepythoncode.com/article/extracting-image-metadata-in-python
#!pip install pillow
#from PIL import Image
#from PIL.ExifTags import TAGS
# path to the image or video
#imagename = "/Users/secchim/Documents/Experiments/VWF_067_MS220522_24hdep/VWF_067_220526_M2_movie1.czi"

# read the image data using PIL
#image = Image.open(imagename)


You should consider upgrading via the '/opt/anaconda3/bin/python -m pip install --upgrade pip' command.


In [2]:
#APPROACH 2

#!pip install czifile
import czifile
from lxml import etree
czi = czifile.CziFile("/Users/secchim/Documents/Experiments/VWF_067_MS220522_24hdep/VWF_067_220526_M2_movie1.czi")

czi_xml_str = czi.metadata()

czi_parsed = etree.fromstring(czi_xml_str)

print(czi_xml_str)

# this returns a list (take first element of list under presumption
# that a file can't have multiple creation dates
acquisition_time = czi_parsed.xpath("//AcquisitionDateAndTime")[0]
position_coordinates = czi_parsed.xpath("//CenterPosition")[0]#how do you do per Scene?



#import pandas as pd
df = [acquisition_time, position_coordinates]

print(df)
#print(acquisition_time.text, position_coordinates.text)#How do you print in a table?


<ImageDocument>
  <Metadata>
    <Experiment Version="1.2">
      <RunMode>OptimizeBeforePerformEnabled,ValidateAndAdaptBeforePerformEnabled,TimeStitchLoopedBlockEnabled</RunMode>
      <BeforeHardwareSetting>Before Exp [AF633, DAPI] Smart</BeforeHardwareSetting>
      <AfterHardwareSetting>After Exp [AF633, DAPI] Smart</AfterHardwareSetting>
      <ExperimentBlockIndex>0</ExperimentBlockIndex>
      <IsSegmented>false</IsSegmented>
      <IsStandardMode>true</IsStandardMode>
      <ImageTransferMode>MemoryMappedAndFileStream</ImageTransferMode>
      <AutoSave IsActivated="false" EnableSingleFileSave="false">
        <StorageFolder>C:\Users\zeiss\Pictures</StorageFolder>
        <Name>VWF_043_MS211118_m1</Name>
        <IsAutoSubFolder>true</IsAutoSubFolder>
        <AlwaysAddFileIndex>true</AlwaysAddFileIndex>
        <IsAutoSubFolderExternal>true</IsAutoSubFolderExternal>
        <IsClosedOnCompletion>false</IsClosedOnCompletion>
        <InitialIndex>1</InitialIndex>
        <Singl

In [24]:
#APPROACH 3 #https://forums.zeiss.com/microscopy/community/viewtopic.php?t=2986

xmlFile = r'/Users/secchim/Downloads'
#image = Zen.Application.Documents.ActiveDocument
image = '/Users/secchim/Documents/Experiments/VWF_067_MS220522_24hdep/VWF_067_220526_M2_movie1.czi'
image.Metadata.SaveAsXml(xmlFile)

AttributeError: 'str' object has no attribute 'Metadata'

In [96]:
#APPROACH 4 https://forum.image.sc/t/extract-image-center-position-from-czi-metadata-in-fiji/42750/7 
#https://forum.image.sc/t/pulling-channel-color-from-czi-metadata/40179/3 

import pandas as pd
import numpy as np
import czifile as zis

filename = r"/Users/secchim/Documents/Experiments/VWF_067_MS220522_24hdep/VWF_067_220526_M2_movie1.czi"

# get CZI object
czi = zis.CziFile(filename)

# parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)

# get the complete scene information #what's good here is name 
allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']#['AcquisitionDateAndTime']
#allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime']
#print(metadatadict_czi[""])


#print(allscenes)
#df=pd.DataFrame(allscenes)
#print(df)


#empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDateAndTime':[],
}

#append values #lists have functions associated with them 

for site in allscenes:
    coordinates=site['CenterPosition'].split(',')
    df['Position_Name'].append('VWF_067_220526_M2_movie1'+'_'+site['Name'])
    df['Position_X'].append(coordinates[0])
    df['Position_Y'].append(coordinates[1])
    #[:-1] will remove the last letter aka Z
    df['AcquisitionDateAndTime'].append(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'])
    
df

pd.DataFrame(df)
saved_df=pd.DataFrame(df)
saved_df.to_csv('/Users/secchim/Documents/metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers



#print(metadatadict_czi)

NameError: name 'os' is not defined

In [104]:
import os

#empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDateAndTime':[],
}

filepath='/Volumes/LoCelso/Secchi Marine'

folder_list=['VWF_067_MS220526_IVMdepletion24h', 'VWF_066_MS220519_IVMdepletion24h','VWF_063_MS220428_IVMdepletion24h',
             'VWF_062_MS220418_IVMdepletion24h','VWF_060_MS220406_IVMpltdepletion24h','VWF_053_MS220208_IVMdepletion',
             'VWF_049_MS220201_IVM_depletion','VWF_048_MS220117_IVM_depletion','VWF_045_MS211208_2h_24h_pltdepletion_IVM',
             'VWF_043_MS211118_onstage_plt_depletion','VWF_039_MS210915_allgenotype','VWF_037_MS210905_IVM_pltdplt_D2_Hoxb5Vwf']
for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files: 
            try:
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                czi = zis.CziFile(filename)
                #print(filename)
                # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']


                #append values #lists have functions associated with them 

                for site in allscenes:
                    coordinates=site['CenterPosition'].split(',')
                    df['Position_Name'].append(file+'_'+site['Name'])
                    df['Position_X'].append(coordinates[0])
                    df['Position_Y'].append(coordinates[1])
                    #[:-1] will remove the last letter aka Z
                    df['AcquisitionDateAndTime'].append(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'])
            except:
                print(f"{file} only has one scene")#putting variable inside string


pd.DataFrame(df)
#saved_df=pd.DataFrame(df)
#saved_df.to_csv('/Users/secchim/Documents/metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers





VWF_067_220526_M2_p9.czi only has one scene
VWF_067_220526_M2_p8.czi only has one scene
VWF_067_220526_M2_p7.czi only has one scene
VWF_067_220526_M2_snap1.czi only has one scene
VWF_067_220526_M2_snap2.czi only has one scene
VWF_067_220526_M2_snap3.czi only has one scene
VWF_067_220526_M2_p5DP.czi only has one scene
VWF_067_220526_M2_snap4.czi only has one scene
VWF_067_220526_M1_p9.czi only has one scene
VWF_067_220527_M1_p11.czi only has one scene
VWF_067_220527_M1_p10.czi only has one scene
VWF_067_220527_M1_p8.czi only has one scene
VWF_067_220527_M1_p12.czi only has one scene
VWF_067_220527_M1_p14bone.czi only has one scene
VWF_067_220527_M1_p15bone.czi only has one scene
VWF_067_220527_M1_p16bone.czi only has one scene
VWF_067_220527_M1_p17dp.czi only has one scene
VWF_067_220527_M1_p4.czi only has one scene
VWF_067_220527_M1_p5.czi only has one scene
VWF_067_220527_M1_snap1.czi only has one scene
VWF_067_220527_M1_p18.czi only has one scene
VWF_067_220527_M1_p19.czi only has on

,Position_Name,Position_X,Position_Y,AcquisitionDateAndTime
0,VWF_067_220526_M2_movie1.czi_P5,32867.5,-14231.5,2022-05-26T10:17:04.025948Z
1,VWF_067_220526_M2_movie1.czi_P7,34948.4,-13378.6,2022-05-26T10:17:04.025948Z
2,VWF_067_220526_M2_movie1.czi_P8,35096.7,-13229.3,2022-05-26T10:17:04.025948Z
3,VWF_067_220526_M2_movie1.czi_P9,34483.5,-12686.6,2022-05-26T10:17:04.025948Z
4,VWF_067_220526_M2_movie2.czi_P5,32867.5,-14231.5,2022-05-26T10:22:39.1986564Z
5,VWF_067_220526_M2_movie2.czi_P7,34948.4,-13378.6,2022-05-26T10:22:39.1986564Z
6,VWF_067_220526_M2_movie2.czi_P8,35096.7,-13273.9,2022-05-26T10:22:39.1986564Z
7,VWF_067_220526_M2_movie2.czi_P9,34483.5,-12686.6,2022-05-26T10:22:39.1986564Z
8,VWF_067_220527_M1_p8zstack.czi_P8k,2720.6,43654.3,2022-05-27T09:57:29.8329243Z
9,VWF_067_220527_M1_p8zstack.czi_P9k,2731.1,43654.5,2022-05-27T09:57:29.8329243Z


In [122]:
#Trying to remove files that have both an unprocessed version and an LSM processed version
#this code works for movies and zstacks that have multiple positions

#import os
import re

#empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDateAndTime':[],
}

filepath='/Volumes/LoCelso/Secchi Marine'

folder_list=['VWF_067_MS220526_IVMdepletion24h']

remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
                    print(file,duplicates[0])

        
        for file in files: 
            if file not in remove_list:
                try:
                    filename=os.path.join(root,file)#looks at the directory used, joins the strings
                    czi = zis.CziFile(filename)
                    #print(filename)
                    # parse the XML into a dictionary
                    metadatadict_czi = czi.metadata(raw=False)


                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']


                    #append values #lists have functions associated with them 

                    for site in allscenes:
                        coordinates=site['CenterPosition'].split(',')
                        df['Position_Name'].append(file+'_'+site['Name'])
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        #[:-1] will remove the last letter aka Z
                        
                        df['AcquisitionDateAndTime'].append(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'])
                except Exception as e:
                    print e
                    print(f"{file} only has one scene")#putting variable inside string
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['Scene']
                    for site in allscenes:
                        coordinates=site['CenterPosition'].split(',')
                        df['Position_Name'].append(file+'_'+site['Name'])
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        #[:-1] will remove the last letter aka Z
                        df['AcquisitionDateAndTime'].append(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'])


test=pd.DataFrame(df)
test
#saved_df=pd.DataFrame(df)
#saved_df.to_csv('/Users/secchim/Documents/metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers



VWF_067_220526_M1_p9-LSM Plus Processing-01.czi VWF_067_220526_M1_p9
VWF_067_220526_M2_p5DP-LSM Plus Processing-02.czi VWF_067_220526_M2_p5DP
VWF_067_220526_M2_p7-LSM Plus Processing-03.czi VWF_067_220526_M2_p7
VWF_067_220526_M2_p8-LSM Plus Processing-04.czi VWF_067_220526_M2_p8
VWF_067_220526_M2_p9-LSM Plus Processing-05.czi VWF_067_220526_M2_p9
VWF_067_220527_M1_p-LSM Plus Processing-06.czi VWF_067_220527_M1_p
VWF_067_220527_M1_p4-LSM Plus Processing-07.czi VWF_067_220527_M1_p4
VWF_067_220527_M1_p5-LSM Plus Processing-08.czi VWF_067_220527_M1_p5
VWF_067_220527_M1_p8-LSM Plus Processing-09.czi VWF_067_220527_M1_p8
VWF_067_220527_M1_p10-LSM Plus Processing-10.czi VWF_067_220527_M1_p10
VWF_067_220527_M1_p11-LSM Plus Processing-11.czi VWF_067_220527_M1_p11
VWF_067_220527_M1_p12-LSM Plus Processing-12.czi VWF_067_220527_M1_p12
VWF_067_220527_M1_p14bone-LSM Plus Processing-13.czi VWF_067_220527_M1_p14bone
VWF_067_220527_M1_p15bone-LSM Plus Processing-14.czi VWF_067_220527_M1_p15bone
VWF_06

,Position_Name,Position_X,Position_Y,AcquisitionDateAndTime
0,VWF_067_220526_M2_movie1.czi_P5,32867.5,-14231.5,2022-05-26T10:17:04.025948Z
1,VWF_067_220526_M2_movie1.czi_P7,34948.4,-13378.6,2022-05-26T10:17:04.025948Z
2,VWF_067_220526_M2_movie1.czi_P8,35096.7,-13229.3,2022-05-26T10:17:04.025948Z
3,VWF_067_220526_M2_movie1.czi_P9,34483.5,-12686.6,2022-05-26T10:17:04.025948Z
4,VWF_067_220526_M2_movie2.czi_P5,32867.5,-14231.5,2022-05-26T10:22:39.1986564Z
5,VWF_067_220526_M2_movie2.czi_P7,34948.4,-13378.6,2022-05-26T10:22:39.1986564Z
6,VWF_067_220526_M2_movie2.czi_P8,35096.7,-13273.9,2022-05-26T10:22:39.1986564Z
7,VWF_067_220526_M2_movie2.czi_P9,34483.5,-12686.6,2022-05-26T10:22:39.1986564Z
8,VWF_067_220527_M1_p8zstack.czi_P8k,2720.6,43654.3,2022-05-27T09:57:29.8329243Z
9,VWF_067_220527_M1_p8zstack.czi_P9k,2731.1,43654.5,2022-05-27T09:57:29.8329243Z


In [372]:
#Trying to remove files that have both an unprocessed version and an LSM processed version
#this code works for movies and zstacks that have multiple positions
#NEW VERSION

#import os
#import re
#import json

#empty dictionary
df1={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
}

df2={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
}

filepath='/Volumes/LoCelso/Secchi Marine'

folder_list=['VWF_067_MS220526_IVMdepletion24h']

remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                czi = zis.CziFile(filename)
                #print(filename)
                # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)

#                 try:
#                     allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
# #error here with S

#                     #append values #lists have functions associated with them 

#                     for site in allscenes:
# #error here that center position is a string requiring interger index
#                         coordinates=site['CenterPosition'].split(',')
#                         df1['Position_Name'].append(file+'_'+site['Name'])
#                         df1['Position_X'].append(coordinates[0])
#                         df1['Position_Y'].append(coordinates[1])
# #                         #[:-1] will remove the last letter aka Z

# #error here with Acquisition date and time 
#                         date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
#                         df1['AcquisitionDate'].append(date[0])
#                         df1['AcquisitionTime'].append(date[1])
# #                        df['AcquisitionTime'].[:-1]
                        
# #                         sizeC = np.int(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['SizeC'])

# #                             if sizeC == 1:
# #                             channels_colors.append(metadatadict_czi['ImageDocument']['Metadata']['DisplaySetting']
# #                             ['Channels']['Channel']['Color'])
# #                             if sizeC > 1:
# #                                 for ch in range(sizeC):
# #                                     channels_colors.append(metadatadict_czi['ImageDocument']['Metadata']['DisplaySetting']
# #                                                               ['Channels']['Channel'][ch]['Color'])

                        
#                 except Exception as e:
#                     print(f"{file} only has one scene")#putting variable inside string
                    
                try:
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
    #when keeping parameter I get a weird error of different array size
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df2['Position_Name'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df2['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df2['AcquisitionDate'].append(date[0])
                            df2['AcquisitionTime'].append(date[1]) 
                        if 'MTBStageAxisY' in i.values():
                            df2['Position_Y'].append(i['Position']['value'])

                 
                except Exception as e:
                    print(f"{file} error")
                    
                    

# #error here with Hardwaresetting
#                     try:
#                         new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
# #when keeping parameter I get a weird error of different array size
# #                        for i in new_list: #i is every dict in the list 
#                            # print(i)
#                             if 'MTBStageAxisX' in i.values() in i.values():
#                                 df2['Position_X'].append(i['Position']['value'],i['Id'])
#                             if 'MTBStageAxisY' in i.values() in i.values():
#                                 df2['Position_Y'].append(i['Position']['value'],i['Id'])

#                             date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
#                             df2['AcquisitionDate'].append(date[0])
#                             df2['AcquisitionTime'].append(date[1])
#                     except:
#                         print(f"{file} gives exception")
                        
                            

#test1=pd.DataFrame(df1)
#print(df2)
test2=pd.DataFrame(df2)
test2
#saved_df=pd.DataFrame(df)
#saved_df.to_csv('/Users/secchim/Documents/metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers




VWF_067_MS220908_M1 (GrabImage).czi error


,Position_Name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime
0,VWF_067_220526_M2_movie1.czi,34483.500,-12686.7,2022-05-26,10:17:04.025948Z
1,VWF_067_220526_M2_snap1.czi,33709.900,-13625.0,2022-05-26,09:34:44.1564688Z
2,VWF_067_220526_M2_snap2.czi,34578.900,-12750.1,2022-05-26,10:06:15.2502142Z
3,VWF_067_220526_M2_snap3.czi,33733.700,-13487.1,2022-05-26,10:09:26.3204531Z
4,VWF_067_220526_M2_snap4.czi,33710.300,-13624.5,2022-05-26,10:14:39.7008037Z
...,...,...,...,...,...
67,VWF_067_220527_M1_snap5.czi,514.599,44557.4,2022-05-27,09:45:04.6787894Z
68,VWF_067_220527_M1_snap6.czi,-139.500,44223.0,2022-05-27,09:49:46.3923098Z
69,VWF_067_220527_M1_snap7.czi,1149.000,43387.9,2022-05-27,08:08:37.6802669Z
70,VWF_067_220527_M1_snap8.czi,-696.800,43781.0,2022-05-27,08:04:36.6685496Z


In [7]:
#filename = r"/Users/secchim/Documents/Experiments/VWF_067_MS220522_24hdep/VWF_067_220526_M2_movie1.czi"   
filename = r"/Volumes/LoCelso/Secchi Marine/VWF_062_MS220418_IVMdepletion24h/tilescans/VWF_062_MS220421_M5_TILESCANpadding-stitched.czi"

czi = zis.CziFile(filename)
                    #print(filename)
                    # parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)



#HOW TO MAKE IT PRETTY/ READABLE 

print(json.dumps(metadatadict_czi, indent=4))

#allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']

# allscenes = metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock'].items()
# print(allscenes)

#print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys())
#results #dict_keys(['Channels', 'Tracks', 'X', 'Y', 'T', 'Z', 'S'])

#allscenes

# date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
# df['AcquisitionDate'].append(date[0])
# df['AcquisitionTime'][:-1]
# test=pd.DataFrame(df)
# test



{
    "ImageDocument": {
        "Metadata": {
            "Experiment": {
                "RunMode": "OptimizeBeforePerformEnabled,ValidateAndAdaptBeforePerformEnabled,TimeStitchLoopedBlockEnabled",
                "BeforeHardwareSetting": "Before Exp [AF633, DAPI] Smart",
                "AfterHardwareSetting": "After Exp [AF633, DAPI] Smart",
                "ExperimentBlockIndex": 0,
                "IsSegmented": false,
                "IsStandardMode": true,
                "ImageTransferMode": "MemoryMappedAndFileStream",
                "AutoSave": {
                    "StorageFolder": "C:\\Users\\zeiss\\Pictures",
                    "Name": "VWF_043_MS211118_m1",
                    "IsAutoSubFolder": true,
                    "AlwaysAddFileIndex": true,
                    "IsAutoSubFolderExternal": true,
                    "IsClosedOnCompletion": false,
                    "InitialIndex": 1,
                    "SingleFileSaveFormat": "JPG",
                    "ConvertTo

In [325]:
filename = r"/Volumes/LoCelso/Secchi Marine/VWF_067_MS220526_IVMdepletion24h/VWF_067_220526_M2_p5DP.czi"

# get CZI object
czi = zis.CziFile(filename)

# parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)
#metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']


df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
}


#print(metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection'])
new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']

for i in new_list: #i is every dict in the list 
   # print(i)
    if 'MTBStageAxisX' in i.values() or 'MTBStageAxisY' in i.values():
        print(i['Position']['value'],i['Id'])

# for i in new_list: #i is every dict in the list 
#     if 'Position' in i.keys():
#         print(i)


#    if 'MTBStageAxis' in i.values():
#        print(i)

#if 'MTBStageAxis' in i['Position'].values():
          #  print(i)
# convert into dictionary of list
# with list as values using pandas dataframe
#pd.DataFrame(new_list).to_dict(orient="list")
#print(new_list)

#print({key: [i[key] for i in new_list] for key in new_list[0]})

# for site in allscenes:
#     ParameterX=site['MTBStageAxisX']
#     df['Position_X'].append(ParameterX)
# print(df)


##parameterCollection is a list - not possible, should be key + value

#coordinates=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']
#coordinates

# print(metadatadict_czi.items())
#print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['T']['Positions'].items())
#channels are in information>image>dimensions


#saved_df=pd.DataFrame(metadatadict_czi)
#saved_df.to_csv('/Users/secchim/Documents/metadata.csv', index=False) #only works for dataframes #otherwise you get row numbers

#'MTBStageAxisX'
#MeasurementPosition



32867.8 MTBStageAxisX
-14231.5 MTBStageAxisY


In [395]:
# centerposX = []
# centerposY = []


# # check if there is a list of scenes
# if isinstance(allscenes, list):
#     for s in range(len(allscenes)):
#         for k, v in allscenes[s].items():
#             print(k, ':', v)
#         # get the SceneCenter Position for all scenes
#         centerposX.append(np.double(allscenes[s]['CenterPosition'].split(',')[0]))
#         centerposY.append(np.double(allscenes[s]['CenterPosition'].split(',')[1]))

        
# # and in case of only one scence (= allscenes is not a list)
# if not isinstance(allscenes, list):
#     for k, v in allscenes.items():
#         print(k, ':', v)
#     centerposX.append(np.double(allscenes['CenterPosition'].split(',')[0]))
#     centerposY.append(np.double(allscenes['CenterPosition'].split(',')[1]))

In [393]:
# !pip install aicspylibczi
# from aicspylibczi import CziFile

# #https://pypi.org/project/aicspylibczi/ 


In [394]:
# filename = r"/Users/secchim/Documents/Experiments/VWF_067_MS220522_24hdep"

# for files in filename: 
#     czi = CziFile(files)

#     # Get the shape of the data, the coordinate pairs are (start index, size)
#     dimensions = czi.get_dims_shape()
    
    
    

In [469]:
#Trying to remove files that have both an unprocessed version and an LSM processed version
#this code works for movies and zstacks that have multiple positions and single position snapshots 


import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
}



filepath='/Volumes/LoCelso/Secchi Marine'

folder_list=['VWF_067_MS220526_IVMdepletion24h']

remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                czi = zis.CziFile(filename)
                #print(filename)
                # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)

                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']


                    for site in allscenes:
                        coordinates=site['CenterPosition'].split(',')
                        df['Position_Name'].append(file+'_'+site['Name'])
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
#                         #[:-1] will remove the last letter aka Z


                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])


                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Position_Name'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1]) 
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])

                    
                else:
                    print(file)

pddf=pd.DataFrame(df)
pddf=pddf.drop_duplicates(subset=['Position_Name'])

#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers



VWF_067_MS220908_M1 (GrabImage).czi


In [401]:
#FOR PADDED TILESCANS

##empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
}



filepath='/Volumes/LoCelso/Secchi Marine/VWF_071_MS220718/VWF_071_MS220718_slide1tilescan.czi'

remove_list=[]



czi = zis.CziFile(filename)
#print(filename)
# parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)




if "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() :

    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
    for i in new_list: #i is every dict in the list 
#                         print(i)
        if 'MTBStageAxisX' in i.values():
            df['Position_Name'].append(file)
#                             print(i['Position']['value'],i['Id'])
            df['Position_X'].append(i['Position']['value'])
            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
            df['AcquisitionDate'].append(date[0])
            df['AcquisitionTime'].append(date[1]) 
        if 'MTBStageAxisY' in i.values():
            df['Position_Y'].append(i['Position']['value'])


else:
    print(file)

pddf=pd.DataFrame(df)
pddf=pddf.drop_duplicates(subset=['Position_Name'])

pddf

,Position_Name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime
0,VWF_067_220527_M1_snap9.czi.xml,34483.5,-12686.7,2022-05-26,10:17:04.025948Z


In [467]:
#FOR PADDED TILESCANS

##empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
}



filepath='/Volumes/LoCelso/Secchi Marine/VWF_071_MS220718/VWF_071_MS220718_slide1tilescan.czi'

remove_list=[]



czi = zis.CziFile(filename)
#print(filename)
# parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)


#For tilescan from a block experiment : Tile information is in ['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']

if "tilescan" in filename and "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() :

    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
    for i in new_list: #i is every dict in the list 
#                         print(i)
        if 'MTBStageAxisX' in i.values():
            df['Position_Name'].append(file)
#                             print(i['Position']['value'],i['Id'])
            df['Position_X'].append(i['Position']['value'])
            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
            df['AcquisitionDate'].append(date[0])
            df['AcquisitionTime'].append(date[1]) 
#             dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
#             df['NZslices'].append(dimensions['SizeZ']) 
#             df['NChannel'].append(dimensions['SizeC'])
#             df['Nframes'].append(dimensions['SizeT'])
#             tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
#             df['Tilecolumn'].append(tile['Columns'])
#             df['Tilerow'].append(tile['Rows'])
#             df['NZslices'].append(tile['Z'])
        if 'MTBStageAxisY' in i.values():
            df['Position_Y'].append(i['Position']['value'])

#     if "Columns" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion'].keys:
#         tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
#         df['Tilecolumn'].append(tile['Columns'])
if "tilescan" in filename: 
    dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
    df['NZslices'].append(dimensions['SizeZ']) 
    df['NChannel'].append(dimensions['SizeC'])
    df['Nframes'].append(dimensions['SizeT'])
    tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
    df['Tilecolumn'].append(tile['Columns'])
    df['Tilerow'].append(tile['Rows'])
    df['NZslices'].append(tile['Z'])

            #ExperimentBlock #TileDimension

else:
    print(file)
    
print(df)

#pddf=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])

#pddf

{'Position_Name': ['VWF_067_220527_M1_snap9.czi.xml'], 'Position_X': [-24371.8], 'Position_Y': [1574.6], 'AcquisitionDate': ['2022-07-18'], 'AcquisitionTime': ['08:56:23.2400533Z'], 'NChannel': [5], 'Nframes': [1], 'NZslices': [10, 2749.541], 'Tilerow': [1], 'Tilecolumn': [4]}


In [38]:

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

czi = zis.CziFile('/Volumes/LoCelso/Secchi Marine/VWF_062_MS220418_IVMdepletion24h/VWF_062_MS220421_M5_TILESCAN.czi')

#print(filename)
# parse the XML into a dictionary

metadatadict_czi = czi.metadata(raw=False)

#print(metadatadict_czi ['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes'].items())

#print(metadatadict_czi ['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection'])

with open ('checkmetadataVWF_062_MS220421_M5_TILESCAN.czi.txt', 'w') as f:
    json.dump(metadatadict_czi, f, indent=4)




In [6]:
#Trying to remove files that have both an unprocessed version and an LSM processed version
#this code works for movies and zstacks that have multiple positions and single position snapshots 
#FINAL VERSION

import os
import re
import json
import pandas as pd
import numpy as np
import czifile as zis

#empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
    'Original_Filename':[]
}



filepath='/Volumes/LoCelso/Secchi Marine'

folder_list=['VWF_043_MS211118_onstage_plt_depletion','VWF_045_MS211208_2h_24h_pltdepletion_IVM', 'VWF_048_MS220117_IVM_depletion','VWF_049_MS220201_IVM_depletion',
'VWF_053_MS220208_IVMdepletion','VWF_060_MS220406_IVMpltdepletion24h','VWF_062_MS220418_IVMdepletion24h','VWF_063_MS220428_IVMdepletion24h','VWF_066_MS220519_IVMdepletion24h','VWF_067_MS220526_IVMdepletion24h' ]
#VWF_062_MS220418_IVMdepletion24h
remove_list=[]


for folder in folder_list:
    filepath=os.path.join(filepath,folder)

    for root, dirs, files in os.walk(filepath): # will open up all the folders, dirs is all the name of the folder it finds, files will contain all the filenames it finds
        for file in files:
            if file.endswith('.czi'):#need to be iterating through file
                duplicates=re.findall(".*(?=-LSM)",file)
                if len(duplicates)>1:
                    remove_list.append(duplicates[0]+'.czi') 
#                    print("test", file,duplicates[0])

        
        for file in files: 
            if file not in remove_list and file.endswith('.czi'):
                #print(file)
                filename=os.path.join(root,file)#looks at the directory used, joins the strings
                try:
                    czi = zis.CziFile(filename)
                except Exception as e:
                    print(f"{file} error {e}")
                    #print(filename)
                    # parse the XML into a dictionary
                metadatadict_czi = czi.metadata(raw=False)


                    

# For multiple positions per file wich should include movies and tilescans
                if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
                    if isinstance(allscenes, dict):
                        coordinates=allscenes['CenterPosition'].split(',')
                        df['Position_Name'].append(file+'_'+allscenes['Name'])#changed from site to allscenes
                        df['Original_Filename'].append(file)
                        df['Position_X'].append(coordinates[0])
                        df['Position_Y'].append(coordinates[1])
                        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                        df['NZslices'].append(dimensions['SizeZ']) 
                        df['NChannel'].append(dimensions['SizeC'])
                        df['Nframes'].append(dimensions['SizeT'])
                        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        df['AcquisitionDate'].append(date[0])
                        df['AcquisitionTime'].append(date[1])

                        # try:
                        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
                        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            
                            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                            if isinstance(tile, list):
                                for i in tile: #going inside the list !!!!
                                    #print(i)
                                    if i['IsUsedForAcquisition']==True:
                                        df['Tilecolumn'].append(i['Columns'])
                                        df['Tilerow'].append(i['Rows'])   
                                    else: 
                                        print(f'{file}IsUsedForAcquisition Not true')                                    

                            #will not work for single positions so the else is required
                            else:
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])
                
                        else:
                            df['Tilecolumn'].append('1')
                            df['Tilerow'].append('1')   
                        # except Exception as e:
                            # print(f"{file} error1 {e}")

                    else:
                            

                        for site in allscenes:#when scenes is a list you have to iterate through each element
                            #print(allscenes,file)
                            coordinates=site['CenterPosition'].split(',')
                            df['Position_Name'].append(file+'_'+site['Name'])
                            df['Original_Filename'].append(file)
                            df['Position_X'].append(coordinates[0])
                            df['Position_Y'].append(coordinates[1])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])

                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                            #except Exception as e:
                            #    print(f"{file} error2")




#For single positions

                elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
                    
                    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
                    for i in new_list: #i is every dict in the list 
#                         print(i)
                        if 'MTBStageAxisX' in i.values():
                            df['Position_Name'].append(file)
                            df['Original_Filename'].append(file)
#                             print(i['Position']['value'],i['Id'])
                            df['Position_X'].append(i['Position']['value'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                        if 'MTBStageAxisY' in i.values():
                            df['Position_Y'].append(i['Position']['value'])
                            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
                            df['NZslices'].append(dimensions['SizeZ']) 
                            df['NChannel'].append(dimensions['SizeC'])
                            df['Nframes'].append(dimensions['SizeT'])
                            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
                            df['AcquisitionDate'].append(date[0])
                            df['AcquisitionTime'].append(date[1])
                            #try:
                            if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                            #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                                tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                                df['Tilecolumn'].append(tile['Columns'])
                                df['Tilerow'].append(tile['Rows'])

                            else:
                                df['Tilecolumn'].append('1')
                                df['Tilerow'].append('1')   
                        # except Exception as e:
                            #   print(f"{file} error3")

                    
                else:
                    print(file)

            
                

for key,value in df.items():
    print(key,len(df[key]))
pddf=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf
pddf.to_csv('/Users/secchim/Documents/VWF_043_045_048_049_053_060_062_063_066_067_IVM_metadata.csv', index=False)


._VWF_043_MS211126_m4-snap1699.czi error not a CZI file
._VWF_043_MS211126_m4-snap1694.czi error not a CZI file
._VWF_043_MS211126_m4-snap1689.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1688.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1687.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1686.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1685.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1684.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1683.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1681.czi error not a CZI file
._VWF_043_MS211126_m4-snap-1680.czi error not a CZI file
._VWF_043_MS211126_m4-pltdepletion_new_position.czi error not a CZI file
._VWF_043_MS211126_m4-P20.czi error not a CZI file
._VWF_043_MS211126_m4-P19.czi error not a CZI file
._VWF_043_MS211126_m4-P17snap.czi error not a CZI file
._VWF_043_MS211126_m4-P17.czi error not a CZI file
._VWF_043_MS211126_m4-P1dp.czi error not a CZI file
._VWF_043_MS211126_m4-P12.czi error not a C

In [41]:
#Trying to remove files that have both an unprocessed version and an LSM processed version
#this code works for movies and zstacks that have multiple positions and single position snapshots 
#testing individual files

#empty dictionary
df={
    'Position_Name':[],
    'Position_X':[],
    'Position_Y':[],
    'AcquisitionDate':[],
    'AcquisitionTime':[],
    'NChannel':[],
    'Nframes':[],
    'NZslices':[],
    'Tilerow':[],
    'Tilecolumn':[],
}



filename='/Volumes/LoCelso/Secchi Marine/VWF_062_MS220418_IVMdepletion24h/VWF_062_MS220421_M5_TILESCAN.czi'

remove_list=[]



czi = zis.CziFile(filename)
#print(filename)
# parse the XML into a dictionary
metadatadict_czi = czi.metadata(raw=False)



        

# For multiple positions per file wich should include movies and tilescans
if "S" in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
    allscenes = metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions']['S']['Scenes']['Scene']
#                     if file == 'VWF_043_MS211125_m4-softwareautofocus_gfpref_1.czi':
#                         print(metadatadict_czi['ImageDocument']['Metadata']['Information']['Image'].items())
    if isinstance(allscenes, dict):
        coordinates=allscenes['CenterPosition'].split(',')
        df['Position_Name'].append(filename+'_'+site['Name'])
        df['Position_X'].append(coordinates[0])
        df['Position_Y'].append(coordinates[1])
        dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
        df['NZslices'].append(dimensions['SizeZ']) 
        df['NChannel'].append(dimensions['SizeC'])
        df['Nframes'].append(dimensions['SizeT'])
        date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
        df['AcquisitionDate'].append(date[0])
        df['AcquisitionTime'].append(date[1])

        # try:
        if "tilescan" in filename.casefold() and "RegionsSetup" in metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups'].keys(): 
        #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
            
            tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
            if isinstance(tile, list):
                for i in tile: #going inside the list !!!!
                    #print(i)
                    if i['IsUsedForAcquisition']==True:
                        df['Tilecolumn'].append(i['Columns'])
                        df['Tilerow'].append(i['Rows'])   
                    else: 
                        print(f'{file}IsUsedForAcquisition Not true')                                    


            else:
                df['Tilecolumn'].append(tile['Columns'])
                df['Tilerow'].append(tile['Rows'])
    
        else:
            df['Tilecolumn'].append('1')
            df['Tilerow'].append('1')   
        # except Exception as e:
            # print(f"{file} error1 {e}")

    else:
        

        for site in allscenes:#when scenes is a list you have to iterate through each element
            #print(allscenes,file)
            coordinates=site['CenterPosition'].split(',')
            df['Position_Name'].append(file+'_'+site['Name'])
            df['Position_X'].append(coordinates[0])
            df['Position_Y'].append(coordinates[1])
            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
            df['NZslices'].append(dimensions['SizeZ']) 
            df['NChannel'].append(dimensions['SizeC'])
            df['Nframes'].append(dimensions['SizeT'])
            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
            df['AcquisitionDate'].append(date[0])
            df['AcquisitionTime'].append(date[1])

            try:
                if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                    tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                    df['Tilecolumn'].append(tile['Columns'])
                    df['Tilerow'].append(tile['Rows'])

                else:
                    df['Tilecolumn'].append('1')
                    df['Tilerow'].append('1')   
            except Exception as e:
                print(f"{file} error2")




#For single positions

elif "HardwareSetting" in metadatadict_czi['ImageDocument']['Metadata'].keys() and "S" not in metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['Dimensions'].keys():
    
    new_list=metadatadict_czi['ImageDocument']['Metadata']['HardwareSetting']['ParameterCollection']
    for i in new_list: #i is every dict in the list 
#                         print(i)
        if 'MTBStageAxisX' in i.values():
            df['Position_Name'].append(file)
#                             print(i['Position']['value'],i['Id'])
            df['Position_X'].append(i['Position']['value'])
            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
        if 'MTBStageAxisY' in i.values():
            df['Position_Y'].append(i['Position']['value'])
            dimensions=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']
            df['NZslices'].append(dimensions['SizeZ']) 
            df['NChannel'].append(dimensions['SizeC'])
            df['Nframes'].append(dimensions['SizeT'])
            date=metadatadict_czi['ImageDocument']['Metadata']['Information']['Image']['AcquisitionDateAndTime'].split('T')
            df['AcquisitionDate'].append(date[0])
            df['AcquisitionTime'].append(date[1])
            try:
                if "tilescan" in filename or "TILESCAN" in filename and "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                #if "ExperimentBlocks" in metadatadict_czi['ImageDocument']['Metadata']['Experiment'].keys(): 
                    tile=metadatadict_czi['ImageDocument']['Metadata']['Experiment']['ExperimentBlocks']['AcquisitionBlock']['SubDimensionSetups']['RegionsSetup']['SampleHolder']['TileRegions']['TileRegion']
                    df['Tilecolumn'].append(tile['Columns'])
                    df['Tilerow'].append(tile['Rows'])

                else:
                    df['Tilecolumn'].append('1')
                    df['Tilerow'].append('1')   
            except Exception as e:
                print(f"{file} error3")

    
else:
    print(file)

                
                

for key,value in df.items():
    print(key,len(df[key]))
pddf=pd.DataFrame(df)
#pddf=pddf.drop_duplicates(subset=['Position_Name'])
#pddf
#pddf.to_csv('/Users/secchim/Documents/VWF_067_MS220909_metadata3.csv', index=False) #only works for dataframes #otherwise you get row numbers

#pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))#fills empty with na
pddf
pddf.to_csv('/Users/secchim/Documents/VWF_062_MS220421_M5_TILESCAN.czi_metadata.csv', index=False)
pddf

Position_Name 1
Position_X 1
Position_Y 1
AcquisitionDate 1
AcquisitionTime 1
NChannel 1
Nframes 1
NZslices 1
Tilerow 1
Tilecolumn 1


,Position_Name,Position_X,Position_Y,AcquisitionDate,AcquisitionTime,NChannel,Nframes,NZslices,Tilerow,Tilecolumn
0,/Volumes/LoCelso/Secchi Marine/VWF_062_MS22041...,-1266.838,45580.833,2022-04-21,09:51:02.239213Z,4,1,10,3,5


In [29]:
folderpath='/Volumes/LoCelso/Secchi Marine/test_folder'
for root,dirs, files in os.walk(folderpath):
    for file in files:
        print(os.path.join(root,file))

/Volumes/LoCelso/Secchi Marine/test_folder/VWF_060_MS220408_M1_P1DP.czi
/Volumes/LoCelso/Secchi Marine/test_folder/test_subfolder/VWF_060_MS220408_M1_P1DP.czi


In [494]:
pddf=pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in df.items() ]))
pddf
pddf.to_csv('/Users/secchim/Documents/VWF_043_MS211118_onstage_plt_depletion_metadata.csv', index=False)